In [1]:
import tensorflow as tf


def corr2d_multi_in(x, k):
    return tf.reduce_sum([corr2d(x, k) for x, k in zip(x, k)], axis=0)

def corr2d(X, K):
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j].assign(tf.reduce_sum(X[i:i + h, j:j + w] * K))
    return Y

x = tf.constant([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                 [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
k = tf.constant([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(x, k)

def corr2d_multi_in_out(x, k):
    return tf.stack([corr2d_multi_in(x, k) for k in k], 0)

k = tf.stack((k, k + 1, k + 2), 0)
k.shape

corr2d_multi_in_out(x, k)

def corr2d_multi_in_out_1x1(x, k):
    c_i, h, w = x.shape
    c_o = k.shape[0]
    x = tf.reshape(x, (c_i, h * w))
    k = tf.reshape(k, (c_o, c_i))
    y = tf.matmul(k, x)
    return tf.reshape(y, (c_o, h, w))

x = tf.random.normal((3, 3, 3), 0, 1)
k = tf.random.normal((2, 3, 1, 1), 0, 1)

y1 = corr2d_multi_in_out_1x1(x, k)
y2 = corr2d_multi_in_out(x, k)

assert float(tf.reduce_sum(tf.abs(y1 - y2))) < 1e-2